In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys
import re

from scipy.interpolate import make_interp_spline, BSpline
from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [3]:
num_sims = 1000
trials =1000
adapt_c = True
data_tuned_epsilon = False
epsilon = .01

In [4]:
def get_size_vector(mc_out):
    """
    Returns the size (rejection probability under the null) for each method,
    using your print order:
      Normal, Two-Step, SW Test, Naive Bootstrap, Pairwise Bootstrap, Shi (2015)
    """
    # Unpack
    reg, twostep, sw, boot1, boot2, sw_test_opt, boot3, shi = mc_out[:8]
    # Take 1 - (no selection rate)
    size_vec = [
        1 - reg[0],
        1 - twostep[0],
        1 - sw[0],
        1 - boot1[0],
        1 - boot3[0],   # boot3=Pairwise, boot1=Naive
        1 - shi[0],
    ]
    return size_vec

def run_null_size_table(sample_sizes, num_sims, trials, epsilon, data_tuned_epsilon, adapt_c,alpha=.05,
                        gen_data=gen_data, num_params=4):
    table = []
    for nobs in sample_sizes:
        setup_shi_ex = lambda yn,xn: setup_shi(yn,xn)
        gen_data_ex = lambda : gen_data(nobs=nobs, num_params=num_params)
        mc_out = vuong_tests_fast.monte_carlo(
            num_sims,
            gen_data,
            setup_shi_ex,
            trials=trials,
            epsilon=epsilon,
            data_tuned_epsilon = data_tuned_epsilon,
            adapt_c = adapt_c,
            print_stuff=False, alpha=alpha
        )
        size_vec = get_size_vector(mc_out)
        table.append([nobs] + [f"{x:.3f}" for x in size_vec])
    # Print as LaTeX table
    print(r'\begin{tabular}{|c|c|c|c|c|c|c|}')
    print(r'\hline')
    print(r'Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline')
    for row in table:
        print(' & '.join(str(y) for y in row)+r' \\')
    print(r'\hline')
    print(r'\end{tabular}')
    return table

# original example from the paper

In [5]:
# Set your globals as needed
sample_sizes = [100, 200, 500]
table = run_null_size_table(
    sample_sizes=sample_sizes,
    num_sims=num_sims,                  # you set this already
    trials=trials,                      # you set this already
    epsilon=epsilon,
    data_tuned_epsilon=data_tuned_epsilon,
    adapt_c=adapt_c,alpha=.05
)

print(table)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
100 & 0.045 & 0.045 & 0.044 & 0.027 & 0.000 & 0.030 \\
200 & 0.041 & 0.041 & 0.040 & 0.021 & 0.000 & 0.024 \\
500 & 0.046 & 0.046 & 0.045 & 0.023 & 0.000 & 0.030 \\
\hline
\end{tabular}
[[100, '0.045', '0.045', '0.044', '0.027', '0.000', '0.030'], [200, '0.041', '0.041', '0.040', '0.021', '0.000', '0.024'], [500, '0.046', '0.046', '0.045', '0.023', '0.000', '0.030']]


In [6]:
# Set your globals as needed
table = run_null_size_table(
    sample_sizes=sample_sizes,
    num_sims=num_sims,                  # you set this already
    trials=trials,                      # you set this already
    epsilon=epsilon,
    data_tuned_epsilon=data_tuned_epsilon,
    adapt_c=adapt_c,alpha=.025
)

print(table)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
100 & 0.023 & 0.023 & 0.023 & 0.013 & 0.000 & 0.014 \\
200 & 0.020 & 0.020 & 0.018 & 0.007 & 0.000 & 0.009 \\
500 & 0.027 & 0.027 & 0.027 & 0.017 & 0.000 & 0.016 \\
\hline
\end{tabular}
[[100, '0.023', '0.023', '0.023', '0.013', '0.000', '0.014'], [200, '0.020', '0.020', '0.018', '0.007', '0.000', '0.009'], [500, '0.027', '0.027', '0.027', '0.017', '0.000', '0.016']]


In [7]:
# Set your globals as needed
table = run_null_size_table(
    sample_sizes=sample_sizes,
    num_sims=num_sims,                  # you set this already
    trials=trials,                      # you set this already
    epsilon=epsilon,
    data_tuned_epsilon=data_tuned_epsilon,
    adapt_c=adapt_c,alpha=.01
)

print(table)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
100 & 0.007 & 0.007 & 0.008 & 0.004 & 0.000 & 0.003 \\
200 & 0.013 & 0.013 & 0.013 & 0.008 & 0.000 & 0.007 \\
500 & 0.010 & 0.010 & 0.010 & 0.007 & 0.000 & 0.005 \\
\hline
\end{tabular}
[[100, '0.007', '0.007', '0.008', '0.004', '0.000', '0.003'], [200, '0.013', '0.013', '0.013', '0.008', '0.000', '0.007'], [500, '0.010', '0.010', '0.010', '0.007', '0.000', '0.005']]


 # Power examples?

In [8]:
def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [9]:
# Set your globals as needed
sample_sizes = [100, 200, 500]
table = run_null_size_table(
    sample_sizes=sample_sizes,
    num_sims=num_sims,                  # you set this already
    trials=trials,                      # you set this already
    epsilon=epsilon,
    data_tuned_epsilon=data_tuned_epsilon,
    adapt_c=adapt_c,alpha=.05, gen_data=gen_data2
)

print(table)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
100 & 1.000 & 1.000 & 1.000 & 1.000 & 0.000 & 1.000 \\
200 & 1.000 & 1.000 & 1.000 & 1.000 & 0.000 & 0.999 \\
500 & 1.000 & 1.000 & 1.000 & 0.998 & 0.000 & 1.000 \\
\hline
\end{tabular}
[[100, '1.000', '1.000', '1.000', '1.000', '0.000', '1.000'], [200, '1.000', '1.000', '1.000', '1.000', '0.000', '0.999'], [500, '1.000', '1.000', '1.000', '0.998', '0.000', '1.000']]


In [10]:
# Set your globals as needed
table = run_null_size_table(
    sample_sizes=sample_sizes,
    num_sims=num_sims,                  # you set this already
    trials=trials,                      # you set this already
    epsilon=epsilon,
    data_tuned_epsilon=data_tuned_epsilon,
    adapt_c=adapt_c,alpha=.025, gen_data=gen_data2
)

print(table)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
100 & 1.000 & 1.000 & 1.000 & 0.999 & 0.000 & 1.000 \\
200 & 1.000 & 1.000 & 1.000 & 1.000 & 0.000 & 1.000 \\
500 & 1.000 & 1.000 & 1.000 & 1.000 & 0.000 & 1.000 \\
\hline
\end{tabular}
[[100, '1.000', '1.000', '1.000', '0.999', '0.000', '1.000'], [200, '1.000', '1.000', '1.000', '1.000', '0.000', '1.000'], [500, '1.000', '1.000', '1.000', '1.000', '0.000', '1.000']]


In [11]:
# Set your globals as needed
table = run_null_size_table(
    sample_sizes=sample_sizes,
    num_sims=num_sims,                  # you set this already
    trials=trials,                      # you set this already
    epsilon=epsilon,
    data_tuned_epsilon=data_tuned_epsilon,
    adapt_c=adapt_c,alpha=.01, gen_data=gen_data2
)

print(table)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & SW Test & Naive Bootstrap & Pairwise Bootstrap & Shi (2015) \\ \hline \hline
100 & 0.997 & 0.997 & 0.996 & 0.994 & 0.000 & 0.995 \\
200 & 0.997 & 0.997 & 0.997 & 0.993 & 0.000 & 0.997 \\
500 & 0.999 & 0.999 & 0.997 & 0.994 & 0.000 & 0.996 \\
\hline
\end{tabular}
[[100, '0.997', '0.997', '0.996', '0.994', '0.000', '0.995'], [200, '0.997', '0.997', '0.997', '0.993', '0.000', '0.997'], [500, '0.999', '0.999', '0.997', '0.994', '0.000', '0.996']]
